In [ ]:
import pandas as pd 
import csv
import requests
import numpy as np
import matplotlib.pyplot as plt

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import uniform, poisson

In [ ]:
# Traitement de la data

test = pd.read_csv('test_data.csv')
train = pd.read_csv('train_data.csv')

y = train.Survived

#on drop  les noms car pas de liens (à mon avis)
feat = ['Name', 'Survived']
X_train = train.drop(columns = feat)
X_test = test.drop(columns = 'Name')


# on supprime ticket car ne sert à rien
X_train = X_train.drop(columns = 'Ticket')
X_test = X_test.drop(columns = 'Ticket')



# one-hot encoder pour Embarked et Sex
featoh = ['Sex', 'Embarked']

OH = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_train = pd.DataFrame(OH.fit_transform(X_train[featoh]))
oh_val = pd.DataFrame(OH.transform(X_test[featoh]))



#remettre les indexes
oh_train.index = X_train.index
oh_val.index = X_test.index


X_train = X_train.drop(columns = featoh)
X_test = X_test.drop(columns = featoh)

# on recolle
X_train_f = pd.concat([X_train, oh_train], axis = 1)
X_test_f = pd.concat([X_test, oh_val], axis = 1)

X_train_f = X_train_f.drop(columns = 'Cabin')
X_test_f = X_test_f.drop(columns = 'Cabin')



In [ ]:
# création du modèle
model = XGBClassifier(random_state = 1)


In [77]:
# Optimisation des paramètres par méthode Bayesienne
p = {
    'n_estimators' : Integer(50, 1000, prior = 'uniform'),
    'max_depth' : Integer(3, 10, prior = 'uniform'),
    'learning_rate' : Real(0.0001, 0.3, prior = 'log-uniform'),
    'reg_alpha' : Real(0.01, 1.0, prior = 'log-uniform'),
    'reg_lambda': Real(0.001, 1.5, prior = 'log-uniform'),
}

opt = BayesSearchCV(XGBClassifier(random_state = 1), p, cv = 3, n_jobs = -1, random_state = 1, n_iter = 50)
opt.fit(X_train_f, y_train)
print(opt.best_params_)

OrderedDict({'learning_rate': 0.002606083634409021, 'max_depth': 3, 'n_estimators': 761, 'reg_alpha': 0.023277942764233707, 'reg_lambda': 0.9729063707072745})


In [ ]:
# On lui donne les meilleurs paramètres et on l'entraîne
m = opt.best_estimator_
m.fit(X_train_f, y)
prds = m.predict(X_test_f)


In [ ]:
# Juste mettre résultat sous forme CSV pour submit le résultat sur Kaggle
output = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': prds.astype(int)
})

output.to_csv('submission.csv', index=False)
print("✅ submission.csv prêt !")

✅ submission.csv prêt !


J'ai obtenue une précision de 81 % avec mon X_val(supprimé ici) et sur le site Kaggle j'obtien 78,4% (petite baisse mais reste solide)